<h1 style="color:DodgerBlue">Индивидуальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта: класс Customer в C#</h2>

----

### Вариант задания 11


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Customer в C#, который будет представлять информацию о
клиентах или покупателях. На основе этого класса разработать 2-3 производных
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из
классов должны быть реализованы новые атрибуты и методы, а также
переопределены некоторые методы базового класса для демонстрации
полиморфизма.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [37]:
using System.Reflection;

public class Customer 
{
    public int CustomerId { get; set; }
    public string Name { get; set; }
    public string Email { get; set; }
    public string Phone { get; set; } 


    public Customer (int customerId, string name, string email, string phone)
    {
        CustomerId = customerId;
        Name = name;
        Email = email;
        Phone = phone;
    }

    public delegate void CustomerHandler(string message);
    public event CustomerHandler CustomerUpdated;

    public virtual void UpdateEmail(string newEmail) 
    {
        Email = newEmail;
        // Вызываем событие после обновления почты
        CustomerUpdated?.Invoke($"Почта обновлена на {newEmail}"); // Invoke - вызывает подписанный метод
    }

    public void SayHello(Customer myCustomer)
    {
        Console.WriteLine($"{Name} приветствует {myCustomer.Name}!");
    }

    public virtual void GetFullName() 
    {
        Console.WriteLine(Name);
    }    

    public virtual void ViewProfile()
    {
        Console.WriteLine($"ID клиента: {CustomerId}; имя: {Name}; электронная почта: {Email}");
    }

    public static Customer operator +(Customer c1, Customer c2)
    {
        // Сложение двух Customer'ов
        return new Customer(c1.CustomerId + c2.CustomerId, c1.Name + " и " + c2.Name, c1.Email + " или " + c2.Email,
         c1.Phone + " и " + c2.Phone);
    }

    public void DisplayGenericCustomer()
    {
        Console.WriteLine($"ID: {CustomerId}, имена покупателей: {Name}, электронные почта для связи: {Email}, мобильные телефоны: {Phone}");
    } 

    public void ClassInfo()
    {
        Type type = this.GetType();

        Console.WriteLine($"Класс: {type.Name}");

        MethodInfo[] methods = type.GetMethods(BindingFlags.DeclaredOnly | BindingFlags.Instance | BindingFlags.Public);

        Console.WriteLine("Список методов:");

        int methodCount = 0;
        foreach (var method in methods)
        {
            if(!method.IsSpecialName)
            {
                Console.WriteLine($"{method.Name} ");
                methodCount ++;
            }
        }

        Console.WriteLine($"Количество реализуемых методов: {methodCount}\n");
    }
}

public interface IVipCustomer 
{
    int LoyaltyPoint { get; set; } 
    void ViewLoyaltyPoint(); // для явной реализации интерфейса  
}

public class VipCustomer : Customer, IVipCustomer 
{
    public int LoyaltyPoint { get; set; }

    public VipCustomer (int customerId, string name, string email, string phone, int loyaltyPoint) 
    : base(customerId, name, email, phone)
    {
        LoyaltyPoint = loyaltyPoint;
    }

    public override void ViewProfile()
    {
        base.ViewProfile();
        Console.WriteLine($"баланс лояльности: {LoyaltyPoint}");
    }

    void IVipCustomer.ViewLoyaltyPoint() // Доступно только для объектов типа IVipCustomer
    {
        Console.WriteLine($"баланс лояльности: {LoyaltyPoint}");
    }
}

public class RegularCustomer : Customer 
{
    public string RegistrationDate { get; set; }

    public RegularCustomer (int customerId, string name, string email, string phone, string registrationDate) 
    : base(customerId, name, email, phone)
    {
        RegistrationDate = registrationDate;
    }

    public override void UpdateEmail(string newEmail)
    {
        Email = newEmail;
        RegistrationDate = DateTime.Today.ToString();
    }

    public void MakeGift(RegularCustomer myCustomer)
    {
        Console.WriteLine($"{Name} дарит подарок для {myCustomer.Name}!");
    }
}


public class CompanyCustomer : Customer 
{
    private readonly VipCustomer _owner; 
    public string GroupName { get; set; }
    
    public CompanyCustomer (int customerId, string name, string email, string phone, string groupName, VipCustomer owner) 
    : base(customerId, name, email, phone)
    {
        GroupName = groupName;
        _owner = owner; // Создание зависимости через констурктор 
    }
    
    public override void GetFullName()
    {
        Console.WriteLine(GroupName);
    }

    public virtual void IPO(bool is_public_company)
    {
        if (is_public_company)
        {
            Console.WriteLine("Данная компания является публичной!");
        }
        else
        {
            Console.WriteLine("Данная компания является частной!");
        }
    }

    public void CheckOwner() // Использование зависимости
    {
        if (_owner != null)
        {
            Console.WriteLine($"Компанией {GroupName} владеет {_owner.Name}."); 
            _owner.ViewProfile();
        }
        else
        {
            Console.WriteLine($"Владелец компании {GroupName} не определен.");
        }
    }
}

public interface IPartnerCompanyCustomer 
{
    int Discount { get; set; }
    void CheckDiscount();
    void IPO(int issue);
}

public class PartnerCompanyCustomer : CompanyCustomer, IPartnerCompanyCustomer 
{
    public int Discount { get; set; }

    public PartnerCompanyCustomer (int customerId, string name, string email, string phone, string groupName, VipCustomer owner, int discount) 
    : base(customerId, name, email, phone, groupName, owner)
    {
        Discount = discount;
    }

    public void CheckDiscount()
    {
        Console.WriteLine($"Скидка компании: {Discount}");
    }

    public void IPO(int issue) // Метод от интерфейса IPartnerCompanyCustomer
    {
        Console.WriteLine($"Эмиссия акций составляет {issue} шт.");

    }
    
    public override void IPO(bool is_public_company) // Метод от класса GroupCustomer
    {
        if (is_public_company)
        {
            Console.WriteLine($"Акции {GroupName} уже котируются на бирже!");
        }
        else
        {
            Console.WriteLine($"{Name} не выпускала акций!");
        }
    }

    
}

public class CustomerCollection<T> where T : Customer // Generic-класс
{
    private List<T> _customers = new List<T>();

    public void Add(T customer)
    {
        _customers.Add(customer);
    }

    public void Remove(T customer)
    {
        _customers.Remove(customer);
    }

    public void ViewProfile()
    {
        foreach (var customer in _customers)
        {
            customer.ViewProfile(); 
        }
    }
}

VipCustomer oleg = new VipCustomer(2, "Oleg", "tnk@gmail.com", "89991234455", 25);
PartnerCompanyCustomer blackRock = new PartnerCompanyCustomer(1, "BlackRock", "blckrck@gmail.com", "89997773344", "BlackRockInvestment", oleg, 20);
CompanyCustomer intel = new CompanyCustomer(1, "huang", "hh@mail.ru", "543-867", "IntelCorp", oleg);


// Исопльзование коллекции
Dictionary<string, Customer> customerDictionary = new Dictionary<string, Customer>();

customerDictionary.Add("oleg", oleg);
customerDictionary.Add("intel", intel);


if (customerDictionary.TryGetValue("intel", out Customer foundCustomer)) // второе - переменная, куда будет сохранено найденное значение
{
    Console.WriteLine("Найденый клиент:");
    foundCustomer.ViewProfile(); 
}
else
{
    Console.WriteLine("Клиент не найден");
}


HashSet<Customer> customerSet = new HashSet<Customer>();

customerSet.Add(oleg);
customerSet.Add(blackRock);
customerSet.Add(intel);

Console.WriteLine(customerSet.Add(blackRock)); // Вернет false, т.к. blackRock уже существует

Console.WriteLine(customerSet.Contains(blackRock)); // true


// Использование делегата
public delegate void DisplayDelegate();

DisplayDelegate displayBlackRock = blackRock.ViewProfile;

displayBlackRock();


// События
// Подписываемся на события с помощью лямбда-выражений
oleg.CustomerUpdated += newEmail => Console.WriteLine($"{newEmail}");

oleg.UpdateEmail("best_company_owner@gmail.com");
oleg.UpdateEmail("best_owner@gmail.com");

Найденый клиент:
ID клиента: 1; имя: huang; электронная почта: hh@mail.ru
False
True
ID клиента: 1; имя: BlackRock; электронная почта: blckrck@gmail.com
Почта обновлена на best_company_owner@gmail.com
Почта обновлена на best_owner@gmail.com
